In [1]:
# import pandas as pd
from bleurt import score
from rouge_score import rouge_scorer
import argparse
import os
import pandas as pd 

2022-02-16 04:35:49.539465: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [2]:
parser = argparse.ArgumentParser()

# generated_path="/home/mbastan/XP/ie_t5/best_model/afterpretrain_pretrain_all_4_8/multipleGeneration"
generated_path='/home/mbastan/XP/ie_t5/best_model/afterpretrain_pretrain_all_4_8/multipleGeneration/beam_10outputs_entail_2sents'
# generated_path='/home/mbastan/XP/ie_t5/best_model/output'
input_file="generated_schemas_beam.txt"
num_generation=10

In [3]:
import re
def find_str(tag,sent):
    try:
        str_found = re.findall("%s(.*?)%s"%(tag,tag), sent)[0].strip()
    except:
        str_found=''
    return str_found

In [4]:
def entity_agreement(true_exp, generated_exp):
 
    true_search = re.search('<re>(.+?)<er>',true_exp)
    try:
        regulator_true = true_search.group(1).strip()
    except:
        regulator_true = ''
    true_search = re.search('<el>(.+?)<le>',true_exp)
    try:
        element_true = true_search.group(1).strip()
    except:
        element_true = ''

    grd_search = re.search('<re>(.+?)<er>',generated_exp)
    try:
        regulator_pred = grd_search.group(1).strip()
    except:
        # print('GE',generated_exp)
        regulator_pred = ''

    grd_search = re.search('<el>(.+?)<le>',generated_exp)
    try:
        element_pred = grd_search.group(1).strip()
    except:
        element_pred = ''
    return [regulator_true,regulator_pred, element_true,element_pred]
path = generated_path
generated_file_beam = open(os.path.join(path, input_file))
used_sentences = open(os.path.join(path,  "generation_used_sents.txt"))
output_file = open(os.path.join(path, "all_generated_filterlong_10sam.txt"),'w')


In [5]:
generated_lines_beam = generated_file_beam.readlines()

used_sents_lines = used_sentences.readlines()
df = pd.DataFrame([],columns=['Sup_Sen','True_Exp','True_lbl','True_reg','True_ele',
        'BM_Exp','BM_lbl','BM_reg','BM_ele','BM_scr'])
        
true_labels = []
bm_labels = []
regs_bm_binary = []
ele_bm_binary = []
bleurt_checkpoint = "/home/mbastan/XP/bleurt/BLEURT-20" #test_checkpoint"#
bm_scores = []
true_exps = []
bm_exps = []
scores_l = []
scores_1 = []
scores_2 = []
scorer = score.BleurtScorer(bleurt_checkpoint)

        



INFO:tensorflow:Reading checkpoint /home/mbastan/XP/bleurt/BLEURT-20.
INFO:tensorflow:Config file found, reading.
INFO:tensorflow:Will load checkpoint BLEURT-20
INFO:tensorflow:Loads full paths and checks that files exists.
INFO:tensorflow:... name:BLEURT-20
INFO:tensorflow:... bert_config_file:bert_config.json
INFO:tensorflow:... max_seq_length:512
INFO:tensorflow:... vocab_file:None
INFO:tensorflow:... do_lower_case:None
INFO:tensorflow:... sp_model:sent_piece
INFO:tensorflow:... dynamic_seq_length:True
INFO:tensorflow:Creating BLEURT scorer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Creating SentencePiece tokenizer.
INFO:tensorflow:Will load model: /home/mbastan/XP/bleurt/BLEURT-20/sent_piece.model.
INFO:tensorflow:SentencePiece tokenizer created.
INFO:tensorflow:Creating Eager Mode predictor.
INFO:tensorflow:Loading model.


2022-02-16 04:35:56.835579: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-02-16 04:35:56.836735: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-02-16 04:35:57.061424: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:07:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2022-02-16 04:35:57.066722: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:0f:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2022-02-16 04:35:57.068409: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 2 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB com

INFO:tensorflow:BLEURT initialized.


In [6]:

for i in range(len(generated_lines_beam)):
    if i%num_generation==0:
        use_sent_line = used_sents_lines[i//num_generation].split('<exp>')
        supporting_set = use_sent_line[0].strip()
    generated_line_beam = generated_lines_beam[i].strip()
    


    if supporting_set.strip() == '':
        continue
    true_explan = use_sent_line[1].strip()

    if true_explan.strip().find('<re>') < 0 or true_explan.strip().find('<el>')<0 or true_explan.strip().find('++++')>0:
        continue
    new_row = {}

    bm_line = generated_line_beam[5:generated_line_beam.find('<end>')].replace('<PAD>','')

#     #### regulator and element ####
    [reg_tru,reg_bm,ele_tru,ele_bm]=entity_agreement(true_explan,bm_line)
    reg_tru = ''.join(reg_tru.lower().split())
    ele_tru = ''.join(ele_tru.lower().split())
    reg_bm = ''.join(reg_bm.lower().split())
    ele_bm = ''.join(ele_bm.lower().split())
    new_row['True_reg'] = reg_tru
    new_row['True_ele'] = ele_tru
    new_row['BM_reg'] =reg_bm
    new_row['BM_ele'] =ele_bm
    if reg_tru == ele_tru or reg_tru=='' or ele_tru=='':
        continue
    regs_bm_binary.append(reg_tru==reg_bm)

    ele_bm_binary.append(ele_tru==ele_bm)

    output_file.write('----- Processing Example %d ----\n'%i)
    output_file.write('----- Supporting Senteces ----\n')
    output_file.write(supporting_set)
    new_row['Sup_Sen']=supporting_set
    output_file.write('----- True Explanation  and Label -----\n')
    output_file.write(true_explan)
    
    new_row['True_Exp'] = true_explan[:-9].strip()#[:-15]
    if 'negative' in new_row['True_Exp'].strip():
        new_row['True_lbl'] = 'negative'
    else:
        new_row['True_lbl'] = 'positive'
    new_row['True_lbl'] = true_explan[-9:].strip()
    true_labels.append(new_row['True_lbl'])


#     ## uncomment if you generate labels as well
    output_file.write('----- Beam Generated Explanation and Label -----\n')
    output_file.write(bm_line) 
    output_file.write('\n')   
    bm_end_ind = bm_line.rfind('.') 
    new_row['BM_Exp'] = bm_line[:-9].strip()
    if 'negative' in bm_line[-9:].strip():
        new_row['BM_lbl'] = 'negative'
    else:
        new_row['BM_lbl'] = 'positive'
    bm_labels.append(new_row['BM_lbl'])
    # print(new_row['BM_lbl'])
    # print(new_row['BM_Exp'])
    # break


#     # ###### BLEURT SCORE ######
    try:
        true_exp_bleurt = new_row['True_Exp'] 
        bm_exp_bleurt = new_row['BM_Exp']

        true_exp_bleurt = true_explan.strip()
        bm_exp_bleurt = bm_line.strip()


       
        true_exps.append(true_exp_bleurt)
        bm_exps.append(bm_exp_bleurt)
       
    except Exception as e: 
        print('err: ',e)
        pass


    df = df.append(new_row,ignore_index=True)
    

scores_bm = scorer.score(references=true_exps, candidates=bm_exps)
df['BM_scr'] = scores_bm

with open(os.path.join(path,'postprocess_out_allsamp.csv'),'w') as f:
        df.to_csv(f,index=False)
output_file.close()





2022-02-16 04:37:47.698944: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.11
2022-02-16 04:37:50.280917: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.11
2022-02-16 04:37:50.292654: I tensorflow/stream_executor/cuda/cuda_blas.cc:1838] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


In [10]:
## Maximum capacity
import pandas as pd
# score_file=pd.read_csv('/home/mbastan/XP/ie_t5/best_model/afterpretrain_pretrain_all_4_8/multipleGeneration/postprocess_out.csv')

# score_file=pd.read_csv('/home/mbastan/XP/ie_t5/best_model/output/postprocess_out_allsamp.csv')
score_file=pd.read_csv(
    '/home/mbastan/XP/ie_t5/best_model/afterpretrain_pretrain_all_4_8/multipleGeneration/beam_10outputs_entail_2sents/postprocess_out_allsamp.csv')
num_generation
import numpy as np
bm_scores=list(score_file.BM_scr)
bleurt_avg=0
for i in range(0,len(bm_scores)-num_generation,num_generation):
    bm_scr10=bm_scores[i:i+num_generation]
    bleurt_max=np.max(np.array(bm_scr10))
    bleurt_avg+=bleurt_max


total_bleurt_score=bleurt_avg/1000
print(total_bleurt_score)

0.5367892569005489


In [8]:
# total_bleurt_score = 0.5367892569005489 for 10 beam outputs
# total_bleurt_score = 0.526252811640501 for 10 sampling outputs
# total_bleurt_score=0.5651176493763924 for 50 beam outputs
num_generation

10

In [40]:
### random selection
import pandas as pd
import random
score_file=pd.read_csv(
    '/home/mbastan/XP/ie_t5/best_model/sampling_output/postprocess_out_allsamp.csv')
# score_file=pd.read_csv(
    # '/home/mbastan/XP/ie_t5/best_model/afterpretrain_pretrain_all_4_8/multipleGeneration/beam_10outputs_entail_2sents/postprocess_out_allsamp.csv')
    # '/home/mbastan/XP/ie_t5/best_model/afterpretrain_pretrain_all_4_8/multipleGeneration/postprocess_out_allsamp_50_beam_selection.csv')

import numpy as np
bm_scores=list(score_file.BM_scr)
bleurt_avg=0
num_generation=10
avg_all=[]
for j in range(100):
    bleurt_avg=0
    for i in range(0,len(bm_scores)-num_generation,num_generation):
        bm_scr10=bm_scores[i:i+num_generation]
        bleurt_max=random.choice(bm_scr10)
        bleurt_avg+=bleurt_max
    total_bleurt_score=bleurt_avg/1000
    avg_all.append(total_bleurt_score)


    
print(sum(avg_all)/len(avg_all))
np.std(avg_all)

0.44786950309798124


0.0019042623626477318

In [39]:
len(bm_scores)

1000

0.00468539383410339